In [ ]:
#Se importa vaex
import vaex
import vaex.jupyter
import numpy as np
import pylab as plt
%matplotlib inline

In [ ]:
#Cargamos los datos que contienen longitud y latitud con el paquete vaex
data = vaex.open('/home/jovyan/work/slides/data/tweetCoordinates.csv')
data

In [ ]:
#Usando vaex se crea una grafica con la ubicacion de los tweets usando longitud y latitud
data.plot_widget(data.longitude, 
               data.latitude, 
               shape=400, 
               limits='minmax',
               f='log1p', 
               colormap='magma')

In [ ]:
# Importamos el paquete pandas
import pandas as pd

In [ ]:
# cargamos el archivo que contiene el total de tweets recolectados por medio de pandas
data2 = pd.read_csv('/home/jovyan/work/slides/data/tweets.csv')
data2

In [ ]:
# Normalizamos las columnas FollowerUser, FollowingUser, TweetsUser, FavoritesTweetsUser
cols_to_norm = data2.columns[6:14]
cols_to_norm

data2[cols_to_norm] = data2[cols_to_norm].transform(lambda value: (value - value.mean()) / value.std())
data2

In [ ]:
# https://github.com/plotly/plotly.py/issues/1664#issuecomment-511773518
import plotly.graph_objects as go
import plotly.io as pio

# Set default renderer
pio.renderers.default = 'notebook+jupyterlab'  #  See [1]
# Set default template
pio.templates['slides'] = go.layout.Template(layout=dict(width=800, height=600))
pio.templates.default = 'plotly+slides'  # See [2]

In [ ]:
# Importing the packages
from plotly.offline import iplot
import cufflinks as cf
cf.go_offline()

In [ ]:
# Segunda gráfica distribución de los diferentes dispositivos para realizar los Tweets 
data2['device'].iplot(kind='hist', bins=5, xTitle='Device',
    linecolor='black', yTitle='Count', title='Device Distribution')

In [ ]:
# Tercera gráfica distribución de los diferentes idiomas usados para realizar los Tweets 
data2['language'].iplot(kind='hist', bins=5, xTitle='Language',
    linecolor='black', yTitle='Count', title='Language Distribution')

In [ ]:
# Cuarta gráfica distribución de los diferentes idiomas usados para realizar los Tweets 
data2['verifieduser'].iplot(kind='hist', bins=5, xTitle='Verified Account',
    linecolor='black', yTitle='Count', title='Verified Account Distribution')

In [ ]:
# Agregamos una nueva columna que hace referencia al sistema operativo del dispositivo donde se realizo el tweet
# iOS se referenciara con un --> 0
# Android se referenciara con un --> 1
# Y con 2 Otros sistemas operativos como lo pueden ser Windows, Mac Os, Linux entre otros.
data2.loc[data2['device'] == 'IPHONE', 'systemdevice'] = 0
data2.loc[data2['device'] == 'ANDROID', 'systemdevice'] = 1
data2.loc[data2['device'] == 'IPAD', 'systemdevice'] = 0
data2.loc[data2['device'] == 'WEB APP', 'systemdevice'] = 2
data2.loc[data2['device'] == 'OTHER', 'systemdevice'] = 2
data2

In [ ]:
# Obtenemos una lista con todas nuestras columnas
cols = list(data2)
# Agregamos la columna "SystemDevice" en la posición 10 de nuestra lista
cols.insert(10, cols.pop(cols.index('systemdevice')))

In [ ]:
# Reordenamos las columnas de la tabla "data2" con el orden generado en la lista "cols"
data2 = data2.loc[:, cols]
data2

In [ ]:
# Quinta gráfica que nos muestra la diferente relacion entre seguidores, personas seguidas, cantidad de tweets realizados
# y cantidad de tweets favoritos
import plotly.express as px
fig = px.parallel_coordinates(data2[data2.columns[6:11]],color='systemdevice',
      color_continuous_scale=[(0, "#1b9e77"),   (0.5, "#d95f02"),
                              (0.5, "#d95f02"),   (1, "#7570b3")])
fig.update_layout(autosize=False,width=800,height=400)
fig.show()

In [ ]:
#Solo dejamos el año de creación de la cuenta.
year = []
for fecha in data2["registeredon"]:
    y = fecha.split()
    year.append(y[len(y)-1])
data2["registeredon"] = year

In [ ]:
# Sexta gráfica distribución del año de creación de las cuentas de los usuarios. 
data2['registeredon'].iplot(kind='hist', bins=20, xTitle='Year',
    linecolor='black', yTitle='Count', title='Year Account Was Created')

In [ ]:
# Formato largo
longTable = pd.melt(data2, id_vars=["date","hashtags","text","user","verifieduser","registeredon","retweet","quote","reply","likes","isquote","originaldate",
                  "originaltext","originaluser","originalverified","originallocation","originalquotecount",
                  "originalreplycount","originalretweetcount","originalfavoritecount","language","device",
                  "location","latitude","longitude","systemdevice"], var_name='myVars')
longTable

In [ ]:
# Septima gráfica HeapMap de las variables FollowerUser, FollowingUser, TweetsUser, FavoritesTweetsUser agrupados
# por el idioma del usuario. 
import plotly.graph_objects as go
fig = go.Figure(data=go.Heatmap(
        z=longTable['value'],
        x=longTable['language'],
        y=longTable['myVars'],
        colorscale='Viridis'));

fig.update_layout(
    xaxis_nticks=80,
    autosize=False,width=1000,height=400);
fig.show()

In [ ]:
# Cargamos nuevamente los datos originales
data2 = pd.read_csv('/home/jovyan/work/slides/data/tweets.csv')
data2

In [ ]:
# Octava gráfica que nos muestra la relacion entre la cantidad de tweets realizados por el usuario
# vs la cantidad de tweets favoritos del usuario teniendo en cuenta si la cuenta es verficada o no y la cantidad 
# usuarios que siguen a la persona.
fig = px.scatter(data2, x="tweetsuser", y="favoritestweetsuser", color="verifieduser",
                 size='followersuser', hover_data=['followersuser'])
fig.show()

In [ ]:
# Generamos la lista de los username o @ de twitter
names = pd.DataFrame(data2, columns=['user'])
text = " ".join(name for name in names["user"])

In [ ]:
#Instalamos Wordcloud
!pip install wordcloud

In [ ]:
#Importamos WordCloud, STOPWORDS, ImageColorGenerator
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Creamos la visualizacion de los username o @ de los usuarios de twitter que tomamos en los datos
wordcloud = WordCloud(background_color="white",width=900,height=600).generate(text)
# Guardamos la imagen en nuestro sistema con el nombre de "names_wordcloud.png"
wordcloud.to_file("/home/jovyan/work/slides/names_wordcloud.png",)

In [ ]:
# Visualizamos algunos de los usuarios que tomamos en la encuesta.
plt.figure(figsize=[15,10])
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
#plt.savefig('/home/jovyan/work/slides/names_wordcloud.png')
plt.show()

In [ ]:
#Guardamos todos los tweets#Guardamos todos los tweets
tableAllTweets = pd.read_csv('/home/jovyan/work/slides/data/tweets.csv')

In [ ]:
## Crear un histograma con columna isquote
tableAllTweets["isquote"]

In [ ]:
# Eliminamos los tweets que no tiene un tweet original (es decir, tweets que son reply, retweet, o quote de otro tweet mas reciente)
import numpy as np

rows_no_quote = np.where(tableAllTweets['isquote']==False)[0]

tweets_with_original_tweet = tableAllTweets.drop(rows_no_quote)



In [ ]:
# Convertimos las columnas de fecha a tipo date 
# para luego verificar la cuanto tardo el usuario del tweet en responder al tweet original
tweets_with_original_tweet['date'] = pd.to_datetime(tweets_with_original_tweet['date'])
tweets_with_original_tweet['originaldate'] = pd.to_datetime(tweets_with_original_tweet['originaldate'])

tweets_with_original_tweet.loc[tweets_with_original_tweet['isquote'] == True, 'responsetime'] = tweets_with_original_tweet['date'] - tweets_with_original_tweet['originaldate']


tweets_with_original_tweet['responsetime'] = tweets_with_original_tweet['responsetime'] / np.timedelta64(1, 'm')

#creamos columna para obtener solo el dia del tweet
tweets_with_original_tweet.loc[tweets_with_original_tweet["date"] != None, 'daydate'] = tweets_with_original_tweet["date"].dt.day

In [ ]:
# tweets_with_original_tweet.columns
#tweets_with_original_tweet['daydate'].tolist()
days = tweets_with_original_tweet["daydate"].astype(str).values.tolist()
days = list(set(days))

days

tweets_with_original_tweet["responsetime"].mean()

In [ ]:
## crear Long Table (tabla larga de datos)
longTable_TWOT = pd.melt(tweets_with_original_tweet, id_vars=['date','user','verifieduser','originaldate', 'originaluser', 'originalverified', 'responsetime'], 
                    var_name='myVars')
longTable_TWOT

In [ ]:
import plotly.express as px

fig = px.line(longTable_TWOT, x='date', y='responsetime', title='Time Series with Rangeslider')

fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [ ]:
#Generaremos grafiaca para determinar la respuesta minima , maxima y el promedio de cada dia que consultamos los datos
#NOTA SOLO FUERON TRES DIAS DE RECOLECTA DE DATOS
import plotly.graph_objects as go

df = tweets_with_original_tweet
x = days

#generamos la lista de los minimos, maximos y los promedios por dia
min_x_day = [999999999, 9999999999,999999999]
max_x_day = [-1,-1,-1]
mean_x_day = [0,0,0]

## Buscamos dependiendo del dia cada valor
for row in df.index:
    i = df['responsetime'][row]
    tweetday = df.daydate[row]
    if str(tweetday) == str(days[0]):
        if min_x_day[0] > i: min_x_day[0] = i
        if max_x_day[0] < i: max_x_day[0] = i
        mean_x_day[0] = (mean_x_day[0]+i) / 2

    elif str(tweetday) == str(days[1]):
        if min_x_day[1] > i: min_x_day[1] = i
        if max_x_day[1] < i: max_x_day[1] = i
        mean_x_day[1] = (mean_x_day[1]+i) / 2
    elif str(tweetday) == str(days[2]):
        if min_x_day[2] > i: min_x_day[2] = i
        if max_x_day[2] < i: max_x_day[2] = i
        mean_x_day[2] = (mean_x_day[2]+i) / 2

#generamos la grafica
fig = go.Figure()
fig.add_trace(go.Bar(x=x, y=min_x_day, name='less response time'))
fig.add_trace(go.Bar(x=x, y=max_x_day, name='longer response time'))
fig.add_trace(go.Bar(x=x, y=mean_x_day, name='average response time'))
# fig.add_trace(go.Bar(x=x, y=range5, name='+ 1 hora'))

fig.update_layout(barmode='relative', title_text='Response times when replying to an original tweet')

fig.update_layout(
   
    xaxis=dict(title='Days'),
    yaxis=dict(
        title='Response Time (Minutes)',
        titlefont_size=16,
        tickfont_size=14,
    )
)

fig.show()